In [ ]:
"""
Bài thực hành 2: Phân tích cảm xúc với LSTM

Mục tiêu:
- Xây dựng mô hình LSTM để phân loại cảm xúc trong đánh giá phim IMDB
- Hiểu cách xử lý dữ liệu văn bản với mạng LSTM
"""

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# PHẦN 1: TẢI VÀ CHUẨN BỊ DỮ LIỆU
print("Đang tải và chuẩn bị dữ liệu...")

# Tham số
max_features = 10000  # Số từ vựng tối đa
maxlen = 200          # Độ dài tối đa của mỗi đánh giá
batch_size = 32
embedding_dim = 128   # Kích thước embedding

# Tải dữ liệu IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(f'Số lượng mẫu huấn luyện: {len(x_train)}')
print(f'Số lượng mẫu kiểm tra: {len(x_test)}')

# Đệm các chuỗi để đảm bảo kích thước đầu vào đồng nhất
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print(f'Kích thước x_train: {x_train.shape}')
print(f'Kích thước x_test: {x_test.shape}')

# Đảm bảo dữ liệu đầu vào và nhãn có đúng kiểu
print(f'Kiểu dữ liệu x_train: {x_train.dtype}')
print(f'Kiểu dữ liệu y_train: {np.array(y_train).dtype}')
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

# Hiển thị một vài ví dụ
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text if i > 3])

print("\nMột vài ví dụ từ tập dữ liệu:")
for i in range(3):
    print(f"Đánh giá {i+1}: {decode_review(x_train[i])[:100]}...")
    print(f"Nhãn: {'Tích cực' if y_train[i] == 1 else 'Tiêu cực'}\n")

# PHẦN 2: XÂY DỰNG MÔ HÌNH LSTM
print("Đang xây dựng mô hình LSTM...")

# Xây dựng mô hình LSTM đơn giản
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Sử dụng Bidirectional LSTM
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Biên dịch mô hình
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# PHẦN 3: HUẤN LUYỆN MÔ HÌNH
print("Đang huấn luyện mô hình...")

# Thêm early stopping để tránh overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Huấn luyện mô hình
epochs = 5  # Giảm số epochs cho demo
history = model.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# PHẦN 4: ĐÁNH GIÁ MÔ HÌNH
print("Đang đánh giá mô hình...")

# Đánh giá trên tập kiểm tra
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f'Độ chính xác trên tập kiểm tra: {test_acc:.4f}')

# Vẽ biểu đồ quá trình huấn luyện
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Huấn luyện')
plt.plot(history.history['val_loss'], label='Xác thực')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Huấn luyện')
plt.plot(history.history['val_accuracy'], label='Xác thực')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# PHẦN 5: PHÂN TÍCH KHẢ NĂNG DỰ ĐOÁN
print("Đang phân tích khả năng dự đoán của mô hình...")

# Dự đoán trên tập kiểm tra
y_pred_prob = model.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Hiển thị confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Tiêu cực', 'Tích cực'],
           yticklabels=['Tiêu cực', 'Tích cực'])
plt.xlabel('Nhãn dự đoán')
plt.ylabel('Nhãn thực tế')
plt.title('Confusion Matrix')
plt.show()

# Hiển thị báo cáo phân loại
print("\nBáo cáo phân loại:")
print(classification_report(y_test, y_pred, target_names=['Tiêu cực', 'Tích cực']))

# PHẦN 6: PHÂN TÍCH MỘT SỐ VÍ DỤ CỤ THỂ
print("Phân tích một số ví dụ cụ thể...")

# Thống kê các dự đoán đúng và sai
correct_indices = [i for i in range(len(y_pred)) if y_pred[i] == y_test[i]]
incorrect_indices = [i for i in range(len(y_pred)) if y_pred[i] != y_test[i]]

print(f"Số lượng dự đoán đúng: {len(correct_indices)}")
print(f"Số lượng dự đoán sai: {len(incorrect_indices)}")

# Hàm hiển thị ví dụ
def display_examples(indices, title):
    print(f"\n{title}:")
    for i in np.random.choice(indices, min(3, len(indices))):
        review = decode_review(x_test[i])
        print(f"Đánh giá: {review[:200]}...")
        print(f"Nhãn thực: {'Tích cực' if y_test[i] == 1 else 'Tiêu cực'}")
        print(f"Dự đoán: {y_pred_prob[i][0]:.4f} -> {'Tích cực' if y_pred[i] == 1 else 'Tiêu cực'}")
        print('---')

# Hiển thị ví dụ dự đoán đúng
display_examples(correct_indices, "Một số ví dụ dự đoán ĐÚNG")

# Hiển thị ví dụ dự đoán sai
display_examples(incorrect_indices, "Một số ví dụ dự đoán SAI")

# PHẦN 7: PHÂN TÍCH DỰ ĐOÁN THEO ĐỘ DÀI ĐẶC TRƯNG
print("Phân tích ảnh hưởng của độ dài đặc trưng đến độ chính xác...")

# Tính độ dài của mỗi đánh giá
review_lengths = np.sum(x_test > 0, axis=1)

# Tạo DataFrame để phân tích
results_df = pd.DataFrame({
    'length': review_lengths,
    'true_label': y_test,
    'predicted_label': y_pred,
    'correct': y_test == y_pred,
    'confidence': y_pred_prob.reshape(-1)
})

# Nhóm theo độ dài và tính độ chính xác
length_bins = [0, 50, 100, 150, 200]
results_df['length_bin'] = pd.cut(results_df['length'], bins=length_bins)
accuracy_by_length = results_df.groupby('length_bin')['correct'].mean()

plt.figure(figsize=(10, 6))
accuracy_by_length.plot(kind='bar')
plt.title('Độ chính xác theo độ dài của đánh giá')
plt.xlabel('Độ dài đánh giá')
plt.ylabel('Độ chính xác')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# PHẦN 8: THỬ DỰ ĐOÁN VỚI ĐẦU VÀO TÙY CHỈNH
print("Thử nghiệm với một số đánh giá tùy chỉnh...")

# Chuẩn bị hàm để xử lý đánh giá tùy chỉnh
def predict_sentiment(review_text):
    # Mã hóa đánh giá
    encoded_review = []
    for word in review_text.lower().split():
        if word in word_index:
            encoded_review.append(word_index[word] + 3)  # +3 vì 0, 1, 2 đã được sử dụng
        else:
            encoded_review.append(2)  # Từ không biết

    # Đệm chuỗi
    padded_review = pad_sequences([encoded_review], maxlen=maxlen)

    # Dự đoán
    prediction = model.predict(padded_review)[0][0]

    return prediction

# Một số đánh giá mẫu để thử nghiệm
sample_reviews = [
    "This movie was amazing! The plot was engaging and the actors delivered outstanding performances.",
    "I didn't like this film at all. The story was boring and the characters were poorly developed.",
    "The movie had some good moments but overall it was just average. I wouldn't watch it again."
]

print("\nDự đoán cảm xúc cho một số đánh giá mẫu:")
for i, review in enumerate(sample_reviews):
    sentiment_score = predict_sentiment(review)
    sentiment = "Tích cực" if sentiment_score > 0.5 else "Tiêu cực"
    print(f"Đánh giá {i+1}: {review}")
    print(f"Dự đoán: {sentiment_score:.4f} -> {sentiment}\n")